# Introduction to Structural Estimation

<font size="3"> ... background material available at <a href="https://github.com/softecon/talks">https://github.com/softecon/talks</a>  </font>

## Usefulness of Structural Econometric Models

* ***Verification*** How accurately does the computational model solve the underlying equations of the model for the quantities of interest?

* ***Validation*** How accurately does the model represent the reality for the quantities of interest?

* ***Uncertainty Quantification*** How do the various sources of error and uncertainty feed into uncertainty in the model-based prediction of the quantities of interest?

<img src='images/venn_diagram.png' width='750' height='750'>

<center> <b><font size='6'> Maximum-Likelihood Estimation, S = 100</font></b></center>
<img src='images/criterion_maximum_likelihood_100.png' width='750' height='750'>

<center> <b><font size='6'> Method of Moments Estimation, S = 100</font></b></center>
<img src='images/criterion_method_moments_100.png' width='750' height='750'>

<center> <b><font size='6'> Maximum-Likelihood Estimation, S = 1,000</font></b></center>
<img src='images/criterion_maximum_likelihood_1000.png' width='750' height='750'>

## Practical Estimation Exercise

<img src='images/documentation.png' width='750' height='750'>